### Missing values

There are many ways data can end up with missing values. For example,

A 2 bedroom house won't include a value for the size of a third bedroom.
A survey respondent may choose not to share his income.
Most machine learning libraries (including scikit-learn) give an error if you try to build a model using data with missing values.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df=pd.read_csv('data/melb_data.csv')
df.head()

Suburb           Address  Rooms Type      Price Method SellerG  \
0  Abbotsford      85 Turner St      2    h  1480000.0      S  Biggin   
1  Abbotsford   25 Bloomburg St      2    h  1035000.0      S  Biggin   
2  Abbotsford      5 Charles St      3    h  1465000.0     SP  Biggin   
3  Abbotsford  40 Federation La      3    h   850000.0     PI  Biggin   
4  Abbotsford       55a Park St      4    h  1600000.0     VB  Nelson   

        Date  Distance  Postcode      ...       Bathroom  Car  Landsize  \
0  3/12/2016       2.5    3067.0      ...            1.0  1.0     202.0   
1  4/02/2016       2.5    3067.0      ...            1.0  0.0     156.0   
2  4/03/2017       2.5    3067.0      ...            2.0  0.0     134.0   
3  4/03/2017       2.5    3067.0      ...            2.0  1.0      94.0   
4  4/06/2016       2.5    3067.0      ...            1.0  2.0     120.0   

   BuildingArea  YearBuilt  CouncilArea Lattitude  Longtitude  \
0           NaN        NaN        Yarra  -37.7996    144.9984   
1          79.0     1900.0        Yarra  -37.8079    144.9934   
2         150.0     1900.0        Yarra  -37.8093    144.9944   
3           NaN        NaN        Yarra  -37.7969    144.9969   
4         142.0     2014.0        Yarra  -37.8072    144.9941   

              Regionname Propertycount  
0  Northern Metropolitan        4019.0  
1  Northern Metropolitan        4019.0  
2  Northern Metropolitan        4019.0  
3  Northern Metropolitan        4019.0  
4  Northern Metropolitan        4019.0  

[5 rows x 21 columns]

In [2]:
# extract column 'Price' from the rest of the dataset
y=df.Price

#we take all columns beside Price
melb_pred=df.drop(['Price'], axis=1)
X = melb_pred.select_dtypes(exclude=['object'])

In [3]:
# split validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

X_train.shape

(10864, 12)

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

#### Approach No. 1 - Drop columns with missing values

Unless most values in the dropped columns are missing, the model loses access to a lot of (potentially useful!) information with this approach. As an extreme example, consider a dataset with 10,000 rows, where one important column is missing a single entry. This approach would drop the column entirely!

In [5]:
#display names of columns containing missing values in X_train

cols_with_miss=[col for col in X_train.columns if X_train[col].isnull().any()]

In [6]:
#drop columns in training and validation data

reduced_X_train=X_train.drop(cols_with_miss, axis=1)
reduced_X_valid=X_valid.drop(cols_with_miss, axis=1)

In [7]:
print("MAE from Approach 1 (drop columns with missing values):")
print(score_dataset(reduced_X_train,reduced_X_valid,y_train,y_valid))

MAE from Approach 1 (drop columns with missing values):
186293.4375727611


#### Approach No. 2 - Imputation

Imputation fills in the missing values with some number. For instance, we can fill in the mean value along each column.

The imputed value won't be exactly right in most cases, but it usually leads to more accurate models than you would get from dropping the column entirely.

In [8]:
from sklearn.impute import SimpleImputer

In [9]:
#imputation
my_imputer=SimpleImputer()
imputed_X_train=pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid=pd.DataFrame(my_imputer.transform(X_valid))

#imputation removed column names; now we put them back

imputed_X_train.columns=X_train.columns
imputed_X_valid.columns=X_valid.columns

In [10]:
print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178741.28717476683


#### Approach No. 3 - An extension to imputation

Imputation is the standard approach, and it usually works well. However, imputed values may be systematically above or below their actual values (which weren't collected in the dataset). Or rows with missing values may be unique in some other way. In that case, your model would make better predictions by considering which values were originally missing.

In this approach, we impute the missing values, as before. And, additionally, for each column with missing entries in the original dataset, we add a new column that shows the location of the imputed entries.

In some cases, this will meaningfully improve results. In other cases, it doesn't help at all.


In [11]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_miss:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

In [12]:
# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
182275.3338856161


In [13]:
print(X_train.shape)

(10864, 12)


In [14]:
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


The training data has 10864 rows and 12 columns, where three columns contain missing data. For each column, less than half of the entries are missing. Thus, dropping the columns removes a lot of useful information, and so it makes sense that imputation would perform better.

As is common, imputing missing values (in Approach 2 and Approach 3) yielded better results, relative to when we simply dropped columns with missing values (in Approach 1).